In [1]:
from mip import *

In [2]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instanceBig1 = {
    "m" : 6,
    "n" : 9,
    "l" : [190, 185, 185, 190, 195, 185],
    "s" : [11, 11, 23, 16, 2, 1, 24, 14, 20],
    "distances" : [
        [0, 200, 119, 28, 180, 77, 145, 61, 123, 87],
        [200, 0, 81, 207, 38, 122, 55, 138, 76, 113],
        [119, 81, 0, 126, 69, 121, 26, 117, 91, 32],
        [28, 207, 126, 0, 187, 84, 152, 68, 130, 94],
        [170, 38, 79, 177, 0, 92, 58, 108, 46, 98],
        [77, 122, 121, 84, 102, 0, 100, 16, 46, 96],
        [145, 55, 26, 152, 58, 100, 0, 91, 70, 58],
        [61, 138, 113, 68, 118, 16, 91, 0, 62, 87],
        [123, 76, 91, 130, 56, 46, 70, 62, 0, 66],
        [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
    ]
}

In [3]:
instance = instanceBig1

In [4]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])
distances = np.array(instance["distances"])
max_distance = np.max(distances, axis=1).sum()

# depot is n+1 (since 0-indexed it's n)

# model = Model(solver_name=CBC)
model = Model()

# x[i,j,k] = 1 if courier i at time k delivers package j
x = model.add_var_tensor((m, n+1, n+2), var_type=BINARY, name="x")

# y[i] is the totale distance travelled by courier i
y = model.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i,j] is 1 if courier i delivers package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

# z[i, j1, j2] is 1 if courier i goes from package j1 to package j2
z = model.add_var_tensor((m, n+1, n+1), var_type=BINARY, name="z")

# w[i,j1,j2,k] is 1 if x[i,j1,k] and x[i,j2,k+1] 
w = model.add_var_tensor((m, n+1, n+1, n+1), var_type=BINARY, name="w")


# each courier start from the depot
model += x[:, n, 0] == 1
# each courier end at the depot
model += x[:, n, n+1] == 1

# t[i,j] 
for i in range(m):
    for j in range(n):
        # model += xsum(x[i,j,k] for k in range(1,n+2)) == t[i,j]
        model += xsum(x[i,j,k] for k in range(n+2)) == t[i,j]

# every package is delivered
for j in range(n):
    model += xsum(t[i,j] for i in range(m)) == 1

# each courier can carry at most l[i] weight
for i in range(m):
    model += xsum(s * t[i,:]) <= l[i]

# # each courier at each time must be in one place
for i in range(m):
    for k in range(n+2):
        model += xsum(x[i,j,k] for j in range(n+1)) == 1

for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):   
            for k in range(n+1):
                model += w[i,j1,j2,k] <= x[i,j1,k]
                model += w[i,j1,j2,k] <= x[i,j2,k+1]
                model += w[i,j1,j2,k] >= x[i,j1,k] + x[i,j2,k+1] - 1

# z[i, j1, j2]
for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):
            if not(j1 == n and j2 == n):
                model += xsum(w[i,j1,j2,k] for k in range(n+1)) == z[i,j1,j2] 
# y[i]
for i in range(m):
    model += xsum(distances[j,k] * z[i,j,k] for j in range(n+1) for k in range(n+1)) == y[i]

# once a courier return to the depot, he can't go anywhere else
for i in range(m):
    for k in range(1,n):
        model += 1- x[i, n, k] + x[i, n, k+1] >= 1


    
v = model.add_var(name="v", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += v >= y[i]


Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [5]:
model.threads = -1

model.objective = minimize(v)

model.optimize()

Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 18801 rows, 7321 columns and 50616 nonzeros
Model fingerprint: 0xf06c8b69
Variable types: 0 continuous, 7321 integer (7314 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 6528 rows and 2400 columns
Presolve time: 1.55s
Presolved: 12273 rows, 4921 columns, 33324 nonzeros
Variable types: 0 continuous, 4921 integer (4914 binary)
Found heuristic solution: objective 1092.0000000
Found heuristic solution: objective 1016.0000000

Concurrent LP optimizer: dua

<OptimizationStatus.OPTIMAL: 0>

In [6]:
print([y[i].x for i in range(m)])

[199.0, 215.0, 226.0, -0.0, -0.0, 209.0]


In [7]:
# print z values
for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):
            # if z[i, j1, j2].x == 1:
                print(f"z[{i}, {j1}, {j2}] = {z[i, j1, j2].x}")

z[0, 0, 0] = 0.0
z[0, 0, 1] = -0.0
z[0, 0, 2] = 0.0
z[0, 0, 3] = 0.0
z[0, 0, 4] = -0.0
z[0, 0, 5] = 0.0
z[0, 0, 6] = -0.0
z[0, 0, 7] = -0.0
z[0, 0, 8] = -0.0
z[0, 0, 9] = -0.0
z[0, 1, 0] = -0.0
z[0, 1, 1] = 0.0
z[0, 1, 2] = -0.0
z[0, 1, 3] = -0.0
z[0, 1, 4] = -0.0
z[0, 1, 5] = -0.0
z[0, 1, 6] = -0.0
z[0, 1, 7] = -0.0
z[0, 1, 8] = -0.0
z[0, 1, 9] = -0.0
z[0, 2, 0] = 0.0
z[0, 2, 1] = -0.0
z[0, 2, 2] = 0.0
z[0, 2, 3] = -0.0
z[0, 2, 4] = 1.0
z[0, 2, 5] = 0.0
z[0, 2, 6] = 0.0
z[0, 2, 7] = -0.0
z[0, 2, 8] = -0.0
z[0, 2, 9] = -0.0
z[0, 3, 0] = 0.0
z[0, 3, 1] = -0.0
z[0, 3, 2] = -0.0
z[0, 3, 3] = 0.0
z[0, 3, 4] = -0.0
z[0, 3, 5] = 0.0
z[0, 3, 6] = -0.0
z[0, 3, 7] = -0.0
z[0, 3, 8] = -0.0
z[0, 3, 9] = -0.0
z[0, 4, 0] = -0.0
z[0, 4, 1] = -0.0
z[0, 4, 2] = -0.0
z[0, 4, 3] = -0.0
z[0, 4, 4] = 0.0
z[0, 4, 5] = -0.0
z[0, 4, 6] = -0.0
z[0, 4, 7] = -0.0
z[0, 4, 8] = -0.0
z[0, 4, 9] = 1.0
z[0, 5, 0] = -0.0
z[0, 5, 1] = -0.0
z[0, 5, 2] = 0.0
z[0, 5, 3] = -0.0
z[0, 5, 4] = -0.0
z[0, 5, 5] = 0.0
z[0, 5, 6

In [8]:
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            if z[i, j1, j2].x == 1:
                print(f"Courier {i} travels from {j1} to {j2}")

Courier 0 travels from 2 to 4
Courier 0 travels from 4 to 9
Courier 0 travels from 9 to 2
Courier 1 travels from 5 to 7
Courier 1 travels from 7 to 9
Courier 1 travels from 8 to 5
Courier 1 travels from 9 to 8
Courier 2 travels from 1 to 6
Courier 2 travels from 6 to 9
Courier 2 travels from 9 to 1
Courier 5 travels from 0 to 3
Courier 5 travels from 3 to 9
Courier 5 travels from 9 to 0


In [9]:
print(model.objective_value)

226.0


In [10]:
# print variable w
for i in range(m):
    for j1 in range(n+1):
        for j2 in range(n+1):
            for k in range(n+1):
                print(f"w[{i}, {j1}, {j2}, {k}] = {w[i, j1, j2, k].x}")

w[0, 0, 0, 0] = 0.0
w[0, 0, 0, 1] = 0.0
w[0, 0, 0, 2] = 0.0
w[0, 0, 0, 3] = 0.0
w[0, 0, 0, 4] = 0.0
w[0, 0, 0, 5] = 0.0
w[0, 0, 0, 6] = 0.0
w[0, 0, 0, 7] = 0.0
w[0, 0, 0, 8] = 0.0
w[0, 0, 0, 9] = 0.0
w[0, 0, 1, 0] = 0.0
w[0, 0, 1, 1] = -0.0
w[0, 0, 1, 2] = -0.0
w[0, 0, 1, 3] = -0.0
w[0, 0, 1, 4] = -0.0
w[0, 0, 1, 5] = -0.0
w[0, 0, 1, 6] = -0.0
w[0, 0, 1, 7] = -0.0
w[0, 0, 1, 8] = -0.0
w[0, 0, 1, 9] = 0.0
w[0, 0, 2, 0] = 0.0
w[0, 0, 2, 1] = -0.0
w[0, 0, 2, 2] = -0.0
w[0, 0, 2, 3] = 0.0
w[0, 0, 2, 4] = -0.0
w[0, 0, 2, 5] = -0.0
w[0, 0, 2, 6] = -0.0
w[0, 0, 2, 7] = -0.0
w[0, 0, 2, 8] = -0.0
w[0, 0, 2, 9] = 0.0
w[0, 0, 3, 0] = 0.0
w[0, 0, 3, 1] = -0.0
w[0, 0, 3, 2] = -0.0
w[0, 0, 3, 3] = 0.0
w[0, 0, 3, 4] = -0.0
w[0, 0, 3, 5] = -0.0
w[0, 0, 3, 6] = -0.0
w[0, 0, 3, 7] = -0.0
w[0, 0, 3, 8] = -0.0
w[0, 0, 3, 9] = 0.0
w[0, 0, 4, 0] = 0.0
w[0, 0, 4, 1] = -0.0
w[0, 0, 4, 2] = -0.0
w[0, 0, 4, 3] = -0.0
w[0, 0, 4, 4] = -0.0
w[0, 0, 4, 5] = -0.0
w[0, 0, 4, 6] = -0.0
w[0, 0, 4, 7] = -0.0
w[0, 0, 4, 

In [11]:
# print variable x
for i in range(m):
    print()
    for j in range(n):
        for k in range(n+2):
            if x[i, j, k].x == 1:
                print(f"x[{i}, {j}, {k}] = {x[i, j, k].x}")


x[0, 2, 1] = 1.0
x[0, 4, 2] = 1.0

x[1, 5, 2] = 1.0
x[1, 7, 3] = 1.0
x[1, 8, 1] = 1.0

x[2, 1, 1] = 1.0
x[2, 6, 2] = 1.0



x[5, 0, 1] = 1.0
x[5, 3, 2] = 1.0


In [12]:
# print variable t
for i in range(m):
    print()
    for j in range(n):
        # if t[i, j].x == 1:
            print(f"t[{i}, {j}] = {t[i, j].x}")


t[0, 0] = 0.0
t[0, 1] = 0.0
t[0, 2] = 1.0
t[0, 3] = 0.0
t[0, 4] = 1.0
t[0, 5] = 0.0
t[0, 6] = 0.0
t[0, 7] = 0.0
t[0, 8] = 0.0

t[1, 0] = 0.0
t[1, 1] = 0.0
t[1, 2] = 0.0
t[1, 3] = 0.0
t[1, 4] = 0.0
t[1, 5] = 1.0
t[1, 6] = 0.0
t[1, 7] = 1.0
t[1, 8] = 1.0

t[2, 0] = 0.0
t[2, 1] = 1.0
t[2, 2] = 0.0
t[2, 3] = 0.0
t[2, 4] = 0.0
t[2, 5] = 0.0
t[2, 6] = 1.0
t[2, 7] = 0.0
t[2, 8] = 0.0

t[3, 0] = 0.0
t[3, 1] = 0.0
t[3, 2] = 0.0
t[3, 3] = 0.0
t[3, 4] = 0.0
t[3, 5] = 0.0
t[3, 6] = 0.0
t[3, 7] = 0.0
t[3, 8] = 0.0

t[4, 0] = 0.0
t[4, 1] = 0.0
t[4, 2] = 0.0
t[4, 3] = 0.0
t[4, 4] = 0.0
t[4, 5] = 0.0
t[4, 6] = 0.0
t[4, 7] = 0.0
t[4, 8] = 0.0

t[5, 0] = 1.0
t[5, 1] = 0.0
t[5, 2] = 0.0
t[5, 3] = 1.0
t[5, 4] = 0.0
t[5, 5] = 0.0
t[5, 6] = 0.0
t[5, 7] = 0.0
t[5, 8] = 0.0
